In [ ]:
import numpy as np 
import pandas as pd 
from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc

from sklearn.model_selection import train_test_split


import tensorflow as tf
from tqdm.autonotebook import tqdm


from keras import Sequential
from keras.callbacks import EarlyStopping

from keras.optimizers import Adam, SGD
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from keras.layers import Lambda, Input, GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model


from keras.preprocessing.image import load_img

In [ ]:
#reading labels csv file
labels = pd.read_csv('../input/dog-breed-identification/labels.csv')
labels.head()

In [ ]:
labels.describe()

In [ ]:
# Lets check one image
from IPython.display import display, Image
Image("../input/dog-breed-identification/train/0a0c223352985ec154fd604d7ddceabd.jpg")

In [ ]:
if len(os.listdir('../input/dog-breed-identification/train/')) == len(labels['id']):
    print('Number of file matches number of actual images!')
else:
    print('Number of file doesnot matches number of actual images!!')

In [ ]:
#Create list of alphabetically sorted labels.
classes = sorted(list(set(labels['breed'])))
n_classes = len(classes)
print('Total unique breed {}'.format(n_classes))

#Map each label string to an integer label.
class_to_num = dict(zip(classes, range(n_classes)))
class_to_num

In [ ]:
labels['breed'].value_counts().plot.bar(figsize=(20,12));

In [ ]:
input_shape = (224,224,3)

def images_to_array(directory, label_dataframe, target_size = input_shape):
    
    image_labels = label_dataframe['breed']
    images = np.zeros([len(label_dataframe), target_size[0], target_size[1], target_size[2]],dtype=np.uint8) #as we have huge data and limited ram memory. uint8 takes less memory
    y = np.zeros([len(label_dataframe),1],dtype = np.uint8)
    
    for ix, image_name in enumerate(tqdm(label_dataframe['id'].values)):
        img_dir = os.path.join(directory, image_name + '.jpg')
        img = load_img(img_dir, target_size = target_size)
        images[ix]=img
        del img
        
        dog_breed = image_labels[ix]
        y[ix] = class_to_num[dog_breed]
    
    y = to_categorical(y)
    
    return images,y

In [ ]:
import time 
t = time.time()

X,y = images_to_array('../input/dog-breed-identification/train', labels[:])

print('runtime in seconds: {}'.format(time.time() - t))

In [ ]:
len(labels)

In [ ]:
n=25

plt.figure(figsize=(20,20))

for i in range(n):
    ax = plt.subplot(5, 5, i+1)
    plt.title(classes[np.where(y[i] ==1)[0][0]])
    plt.imshow(X[i].astype('int32'))

In [ ]:
# Hyperparameters
batch_size= 128
epochs=50
learn_rate=.001
sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)
adam=Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None,  amsgrad=False)

In [ ]:
#function to extract features from the dataset by a given pretrained model
img_size = (224,224,3)

def get_features(model_name, model_preprocessor, input_size, data):

    input_layer = Input(input_size)
    preprocessor = Lambda(model_preprocessor)(input_layer)
    base_model = model_name(weights='imagenet', include_top=False,
                            input_shape=input_size)(preprocessor)
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    
    #Extract feature.
    feature_maps = feature_extractor.predict(data, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    return feature_maps

### Model VGG16

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
vgg16_preprocessor = preprocess_input
vgg_features = get_features(VGG16,
                            vgg16_preprocessor,
                            img_size  , X)

In [ ]:
#Prepare Deep net

model_vgg = Sequential()
model_vgg.add(Dense(1028, input_shape=(vgg_features.shape[1], ),
                       kernel_initializer = 'he_uniform', 
                       kernel_regularizer = None,
                       kernel_constraint = 'MaxNorm',
                       activation = 'relu')) 
model_vgg.add(Dropout(0.7,input_shape=(vgg_features.shape[1],)))
model_vgg.add(Dense(n_classes,activation= 'softmax'))

model_vgg.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

#Training the model. 
history_VGG = model_vgg.fit(vgg_features, y,batch_size=batch_size, epochs=epochs, validation_split=0.2)

In [ ]:
# summarize history for accuracy
plt.plot(history_VGG.history['accuracy'])
plt.plot(history_VGG.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_VGG.history['loss'])
plt.plot(history_VGG.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Model ResNet50

In [ ]:
from keras.applications.resnet50 import ResNet50,preprocess_input
resnet50_preprocessor = preprocess_input
resnet_features = get_features(ResNet50,
                               resnet50_preprocessor,
                               img_size, X)

In [ ]:
#Prepare Deep net

model_resnet = Sequential()
model_resnet.add(Dense(1028, input_shape=(resnet_features.shape[1], ),
                       kernel_initializer = 'he_uniform', 
                       kernel_regularizer = None,
                       kernel_constraint = 'MaxNorm',
                       activation = 'relu')) 
model_resnet.add(Dropout(0.7,input_shape=(resnet_features.shape[1],)))
model_resnet.add(Dense(n_classes,activation= 'softmax'))

model_resnet.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Training the model. 
history_resnet = model_resnet.fit(resnet_features, y,
            batch_size=batch_size,
            epochs=epochs,
            validation_split=0.2)

In [ ]:
# summarize history for accuracy
plt.plot(history_resnet.history['accuracy'])
plt.plot(history_resnet.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_resnet.history['loss'])
plt.plot(history_resnet.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Model DenseNet169

In [ ]:
from keras.applications.densenet import DenseNet169,preprocess_input
densenet169_preprocessor = preprocess_input
densenet_features = get_features(DenseNet169,
                                 densenet169_preprocessor,
                                 img_size, X)

In [ ]:
#Prepare Deep net

model_densenet = Sequential()
model_densenet.add(Dense(1028, input_shape=(densenet_features.shape[1],),
                         kernel_initializer = 'he_uniform', 
                         kernel_regularizer = None,
                         kernel_constraint = 'MaxNorm',
                         activation = 'relu')) 
model_densenet.add(Dropout(0.7,input_shape=(densenet_features.shape[1],)))
model_densenet.add(Dense(n_classes,activation= 'softmax'))

model_densenet.compile(optimizer=adam,
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

#Training the model. 
history_densenet = model_densenet.fit(densenet_features, y,
                                      batch_size=batch_size,
                                      epochs=epochs,
                                      validation_split=0.2)


In [ ]:
# summarize history for accuracy
plt.plot(history_densenet.history['accuracy'])
plt.plot(history_densenet.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_densenet.history['loss'])
plt.plot(history_densenet.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Model MobileNetV2

In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input
mobilenetV2_preprocessor = preprocess_input
mobilenetV2_features = get_features(MobileNetV2,
                                    mobilenetV2_preprocessor,
                                    img_size  , X)

In [ ]:
model_mobilenet = Sequential()
model_mobilenet.add(Dense(1028, input_shape=(mobilenetV2_features.shape[1], ),
                    kernel_initializer = 'he_uniform', 
                    kernel_regularizer = None,
                    kernel_constraint = 'MaxNorm',
                    activation = 'relu')) 
model_mobilenet.add(Dropout(0.7,input_shape=(mobilenetV2_features.shape[1],)))
model_mobilenet.add(Dense(n_classes,activation= 'softmax'))

model_mobilenet.compile(optimizer=adam,
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])

history_mobilenet = model_mobilenet.fit(mobilenetV2_features, y,
                              batch_size=batch_size,
                              epochs=epochs,
                              validation_split=0.2)

In [ ]:
# summarize history for accuracy
plt.plot(history_mobilenet.history['accuracy'])
plt.plot(history_mobilenet.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_mobilenet.history['loss'])
plt.plot(history_mobilenet.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Compare Models

In [ ]:
# Train Loss for different models
plt.plot(history_VGG.history['loss'])
plt.plot(history_resnet.history['loss'])
plt.plot(history_densenet.history['loss'])
plt.plot(history_mobilenet.history['loss'])
plt.title('Train Loss for different models')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['VGG16', 'ResNet50', 'DenseNet169', 'MobileNetV2'], loc='upper left')
plt.savefig('./train_loss.png')
plt.show()
# Test Loss for different models
plt.plot(history_VGG.history['val_loss'])
plt.plot(history_resnet.history['val_loss'])
plt.plot(history_densenet.history['val_loss'])
plt.plot(history_mobilenet.history['val_loss'])
plt.title('Test Loss for different models')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['VGG16', 'ResNet50', 'DenseNet169', 'MobileNetV2'], loc='upper left')
plt.savefig('./test_loss.png')
plt.show()

# Train Accuracy for different models
plt.plot(history_VGG.history['accuracy'])
plt.plot(history_resnet.history['accuracy'])
plt.plot(history_densenet.history['accuracy'])
plt.plot(history_mobilenet.history['accuracy'])
plt.title('Train Accuracy for different models')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['VGG16', 'ResNet50', 'DenseNet169', 'MobileNetV2'], loc='upper left')
plt.savefig('./train_accuracy.png')
plt.show()

# Test Accuracy for different models
plt.plot(history_VGG.history['val_accuracy'])
plt.plot(history_resnet.history['val_accuracy'])
plt.plot(history_densenet.history['val_accuracy'])
plt.plot(history_mobilenet.history['val_accuracy'])
plt.title('Test Accuracy for different models')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['VGG16', 'ResNet50', 'DenseNet169', 'MobileNetV2'], loc='upper left')
plt.savefig('./test_accuracy.png')
plt.show()


In [ ]:
accuracy_test = [np.array(history_VGG.history['val_accuracy']).mean(),
                  np.array(history_resnet.history['val_accuracy']).mean(),
                  np.array(history_densenet.history['val_accuracy']).mean(),
                  np.array(history_mobilenet.history['val_accuracy']).mean()
                 ]
accuracy_train = [np.array(history_VGG.history['accuracy']).mean(),
                  np.array(history_resnet.history['accuracy']).mean(),
                  np.array(history_densenet.history['accuracy']).mean(),
                  np.array(history_mobilenet.history['accuracy']).mean()
                 ]
loss_test = [np.array(history_VGG.history['val_loss']).mean(),
             np.array(history_resnet.history['val_loss']).mean(),
             np.array(history_densenet.history['val_loss']).mean(),
             np.array(history_mobilenet.history['val_loss']).mean()
            ]
loss_train = [np.array(history_VGG.history['loss']).mean(),
              np.array(history_resnet.history['loss']).mean(),
              np.array(history_densenet.history['loss']).mean(),
              np.array(history_mobilenet.history['loss']).mean()
             ]

df = pd.DataFrame({
    'Model':['VGG16','ResNet50','DenseNet169','MobileNetV2'],
    'Loss (Train)':loss_train,
    'Loss (Test)':loss_test,
    'Accuracy (Train)':accuracy_train,
    'Accuracy (Test)': accuracy_test,
})
df

In [ ]:
#Function to read images from test directory

def images_to_array_test(test_path, img_size = (224,224,3)):
    test_filenames = [test_path + fname for fname in os.listdir(test_path)]

    data_size = len(test_filenames)
    images = np.zeros([data_size, img_size[0], img_size[1], 3], dtype=np.uint8)
    
    
    for ix,img_dir in enumerate(tqdm(test_filenames)):
        img = load_img(img_dir, target_size = img_size)
        images[ix]=img
        del img
    print('Ouptut Data Size: ', images.shape)
    return images

test_data = images_to_array_test('../input/dog-breed-identification/test/', img_size)

In [ ]:
#Extract test data features.
def extact_features_vgg(data):
    vgg_features = get_features(VGG16, vgg16_preprocessor, img_size, data)
    return vgg_features
def extact_features_resnet(data):
    resnet_features = get_features(ResNet50, resnet50_preprocessor, img_size, data)
    return resnet_features
def extact_features_densenet(data):
    densenet_features = get_features(DenseNet169, densenet169_preprocessor, img_size, data)
    return densenet_features
def extact_features_mobilenet(data):
    mobilenetV2_features = get_features(MobileNetV2, mobilenetV2_preprocessor, img_size, data)
    return mobilenetV2_features


In [ ]:
test_df = pd.DataFrame({
    'Models': ['VGG16', 'ResNet50', 'DenseNet169', 'MobileNetV2']
})
test_df = test_df.set_index('Models')
dog_names = ["Pomeranian", "Siberian Husky", "Irish Setter", "Danua", "Chihuahua", "Newfoundland", "Maltese ", "Bullmastiff", "Cairn Terrier", "Norwich Terrier"]
test_df = pd.concat([test_df,pd.DataFrame(columns = dog_names)])
test_df

In [ ]:
print('-------------------------------Testing VGG16 Model-------------------------------')
result = []
for j in range(1,11):
    img_g = load_img('../input/mydogsample/dog%s.jpg'%j,target_size = img_size)
    img_g = np.expand_dims(img_g, axis=0) 

    test_features = extact_features_vgg(img_g)
    pred = model_vgg.predict(test_features)
    result.append(classes[np.argmax(pred[0])] + " " + str(round(np.max(pred[0]))* 100)  + "%")
test_df.loc['VGG16'] = result

print('-------------------------------Testing ResNet50 Model-------------------------------')
result = []
for j in range(1,11):
    img_g = load_img('../input/mydogsample/dog%s.jpg'%j,target_size = img_size)
    img_g = np.expand_dims(img_g, axis=0) 

    test_features = extact_features_resnet(img_g)
    pred = model_resnet.predict(test_features)
    result.append(classes[np.argmax(pred[0])] + " " + str(round(np.max(pred[0]))* 100)  + "%")
test_df.loc['ResNet50'] = result

print('-------------------------------Testing DenseNet169 Model-------------------------------')
result = []
for j in range(1,11):
    img_g = load_img('../input/mydogsample/dog%s.jpg'%j,target_size = img_size)
    img_g = np.expand_dims(img_g, axis=0) 

    test_features = extact_features_densenet(img_g)
    pred = model_densenet.predict(test_features)
    result.append(classes[np.argmax(pred[0])] + " " + str(round(np.max(pred[0]))* 100)  + "%")
test_df.loc['DenseNet169'] = result

print('-------------------------------Testing MobileNetV2 Model-------------------------------')
result = []
for j in range(1,11):
    img_g = load_img('../input/mydogsample/dog%s.jpg'%j,target_size = img_size)
    img_g = np.expand_dims(img_g, axis=0) 

    test_features = extact_features_mobilenet(img_g)
    pred = model_mobilenet.predict(test_features)
    result.append(classes[np.argmax(pred[0])] + " " + str(round(np.max(pred[0]))* 100)  + "%")
test_df.loc['MobileNetV2'] = result
result = []

In [ ]:
test_df